In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [4]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fixation.csv')
data_trial_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fixation.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fixation = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fixation.csv')
data_trial_choice = pd.read_csv(
    r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_subject = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_subject.csv') \
    .rename(columns=
            {
                'chosenAmount': 'bonus_pay',
                'chosenDelay': 'bonus_delay'
            }
           )    

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fix)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fix)],
        [len(data_trial_choice)],
        [len(data_subject)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fixation', 'data_et_choice', 
           'data_trial', 'data_trial_fixation', 'data_trial_choice', 
           'data_subject'])
print(overview)

                     length
data_et              618566
data_et_fixation     160870
data_et_choice       156004
data_trial            30362
data_trial_fixation  160870
data_trial_choice      4270
data_subject             40


# Approve subjects

In [5]:
path = 'data_prolific'
subject_files = os.listdir(path)
for i in range(0, len(subject_files)):
    thisSubject = open(path + "/" + subject_files[i]).read()
    if thisSubject.find('5edc20443467e28ec4e30f93') > (-1):
        print(subject_files[i])

In [6]:
if 'trial_length' in data_subject.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

    data_subject = data_subject.merge(output, on=['run_id', 'prolificID'], how='left')

In [7]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [8]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_subject = data_subject.merge(output, on='run_id', how='left')

## These subjects await review

In [9]:
data_subject.loc[data_subject['status']=='AWAITING REVIEW', 
                 ['run_id', 'prolificID', 'trial_length',
                 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
                 'm_choseTop']
                ].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [10]:
data_subject.columns

Index(['chinFirst', 'choiceTask_amountLeftFirst', 'bonus_pay', 'bonus_delay',
       'run_id', 'prolificID', 'age', 'browliner', 'browser',
       'browser_version', 'degree', 'device', 'ethnic', 'eyeliner',
       'eyeshadow', 'gender', 'glasses', 'keptHead', 'masquara',
       'optionalNote', 'platform', 'platform_version', 'sight', 'triedChin',
       'user_agent', 'vertPosition', 'webcam_fps', 'webcam_height',
       'webcam_label', 'webcam_width', 'session_id', 'status',
       'started_datetime', 'completed_date_time', 'time_taken', 'age_2',
       'num_approvals', 'num_rejections', 'prolific_score',
       'reviewed_at_datetime', 'entered_code', 'Country of Birth',
       'Current Country of Residence', 'Employment Status', 'First Language',
       'Nationality', 'Sex', 'Student Status', 'date', 'x_count',
       'trial_duration_exact', 'choseLL', 'fps', 'offset', 'offset_chin_0',
       'offset_chin_1', 'precision', 'precision_chin_0', 'precision_chin_1',
       'trial_length',

In [11]:
if 'choice_rt' in data_subject.columns:
    print('choice_rt already added!')
else: 
    grouped = data_trial.loc[data_trial['trial_type']=='eyetracking-choice', :] \
        .groupby(['prolificID'])['trial_duration_exact'].mean() \
        .reset_index() \
        .rename(columns={'trial_duration_exact': 'choice_rt'})
    data_subject = data_subject.merge(grouped, on='prolificID', how='left')

data_subject.loc[
    data_subject['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 'choice_rt', 'bonus_pay', 'bonus_delay']
] \
    .sort_values(by='run_id')

,run_id,prolificID,webcam_fps,choice_rt,bonus_pay,bonus_delay


# Basic payment

In [12]:
data_pay = data_subject.loc[data_subject['status']=='APPROVED', 
                            [
                                'run_id', 'prolificID', 
                                'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
                                'Nationality', 'Sex',
                                'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
                                'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
                                'bonus_pay', 'bonus_delay'
                            ]
                           ]

data_pay['basic_pay'] = 2.25

# Bonus payment

In [13]:
data_pay[['run_id', 'prolificID', 'bonus_pay']]

,run_id,prolificID,bonus_pay
0,103,600063f2943eab0acc812ed8,4.5
1,11,5c5684ef9d244c0001b29f1e,3.5
2,126,5c0e4e3d87e876000151cfec,4.5
3,128,5f01a0893963d02ea1d65d12,4.5
4,13,5b4e65de09f5af0001080f45,5.0
5,130,5fea19869bf07d97b4a87ae8,4.0
6,131,5e53a7b4d9a7ea3a2b062da0,3.5
7,19,5f860a189bc695131f22ec9c,4.5
8,24,5f77a902fc647a327b77004a,3.0
9,25,5cf0ccf488abf10019fb775d,4.5


## Replace empty values with the best bonus reward

In [14]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
        ]
    ), ['bonus_pay', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay[['bonus_pay', 'bonus_delay']].isnull().any(axis=1), :]

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,Country of Birth,entered_code,session_id,started_datetime,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay


## What bonus is due today? 

In [17]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)
data_pay['bonus_pay'].unique()

array([4.5, 3.5, 5. , 4. , 3. , 2.5, 0.5, 2. ])

In [18]:
data_pay['reward_pound'] = data_pay['bonus_pay'] * 0.75

In [19]:
data_pay['due_on'] = data_pay['completed_date'] +  data_pay['bonus_delay'].map(datetime.timedelta) 

data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'completed_date', 'bonus_pay', 'bonus_delay', 
                   'reward_pound', 'due_on'
               ]
              ]. sort_values(by='due_on')

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1268: PerformanceWarning: Adding/subtracting array of DateOffsets to TimedeltaArray not vectorized
  PerformanceWarning,


,prolificID,run_id,Nationality,Sex,completed_date,bonus_pay,bonus_delay,reward_pound,due_on
10,5f561a95aa1c4ea13672f138,28,Italy,Male,2021-01-18,2.5,0.0,1.875,2021-01-18
1,5c5684ef9d244c0001b29f1e,11,United States,Male,2021-01-18,3.5,0.0,2.625,2021-01-18
8,5f77a902fc647a327b77004a,24,United Kingdom,Female,2021-01-18,3.0,0.0,2.250,2021-01-18
11,5eeaa0ffaa6af11cf32ce057,30,Poland,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
14,5f0cb319d29147695796a208,37,Italy,Male,2021-01-18,3.0,0.0,2.250,2021-01-18
13,5f8472685956c40c720d0936,36,Italy,Male,2021-01-18,0.5,0.0,0.375,2021-01-18
34,5f256074297aac1d8ef7382d,91,Hungary,Male,2021-01-19,2.5,0.0,1.875,2021-01-19
27,5cfb9f6f288ca80001169952,67,Poland,Male,2021-01-19,2.0,0.0,1.500,2021-01-19
17,5fcbb3066829ff1a601fe080,42,United Kingdom,Female,2021-01-18,5.0,1.0,3.750,2021-01-19
5,5fea19869bf07d97b4a87ae8,130,Spain,Male,2021-01-19,4.0,0.0,3.000,2021-01-19


In [20]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), :]
bonus_due_today['reward_pound'] = bonus_due_today['reward_pound'].round(2)
bonus_due_today

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,started_datetime,completed_date_time,time_taken,bonus_pay,bonus_delay,basic_pay,completed_date,date,reward_pound,due_on


In [21]:
bonus_due_today.loc[: , ['prolificID', 'reward_pound']] \
    .to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/bonusToday.csv', 
            index=False, 
            header=False
           )
print(bonus_due_today['due_on'].unique())

[]


# Total costs

In [22]:
data_pay['basic_cost'] = data_pay['basic_pay'] * 1.4
data_pay['total_cost'] = (data_pay['basic_pay'] + data_pay['bonus_pay']) * 1.4
print('Basic Costs: ' + str(sum(data_pay['basic_cost'])))
print('Total Costs: ' + str(sum(data_pay['total_cost'])))
data_pay

Basic Costs: 126.00000000000009
Total Costs: 349.29999999999984


,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,time_taken,bonus_pay,bonus_delay,basic_pay,completed_date,date,reward_pound,due_on,basic_cost,total_cost
0,103,600063f2943eab0acc812ed8,2001.0,Italy,Italy,NaN,Italy,Male,APPROVED,2021-01-19 10:58:01.081000,...,1760.968,4.5,180.0,2.25,2021-01-19,2021-01-19,3.375,2021-07-18,3.15,9.45
1,11,5c5684ef9d244c0001b29f1e,1991.0,Hong Kong,United States,Chinese,United States,Male,APPROVED,2021-01-18 10:53:28.738000,...,1227.532,3.5,0.0,2.25,2021-01-18,2021-01-18,2.625,2021-01-18,3.15,8.05
2,126,5c0e4e3d87e876000151cfec,1991.0,Poland,Poland,Polish,Poland,Female,APPROVED,2021-01-19 12:36:10.712000,...,1912.745,4.5,90.0,2.25,2021-01-19,2021-01-19,3.375,2021-04-19,3.15,9.45
3,128,5f01a0893963d02ea1d65d12,1994.0,Germany,Greece,Greek,Greece,Male,APPROVED,2021-01-19 12:36:47.163000,...,2288.987,4.5,7.0,2.25,2021-01-19,2021-01-19,3.375,2021-01-26,3.15,9.45
4,13,5b4e65de09f5af0001080f45,2001.0,Poland,Poland,Polish,Poland,Male,APPROVED,2021-01-18 10:56:07.011000,...,1692.194,5.0,30.0,2.25,2021-01-18,2021-01-18,3.750,2021-02-17,3.15,10.15
5,130,5fea19869bf07d97b4a87ae8,2001.0,Spain,Spain,NaN,Spain,Male,APPROVED,2021-01-19 13:24:23.588000,...,1349.099,4.0,0.0,2.25,2021-01-19,2021-01-19,3.000,2021-01-19,3.15,8.75
6,131,5e53a7b4d9a7ea3a2b062da0,NaN,United Kingdom,United Kingdom,English,United Kingdom,Female,APPROVED,2021-01-19 13:24:22.246000,...,1415.591,3.5,0.0,2.25,2021-01-19,2021-01-19,2.625,2021-01-19,3.15,8.05
7,19,5f860a189bc695131f22ec9c,2000.0,Poland,Poland,DATA EXPIRED,Poland,Male,APPROVED,2021-01-18 10:57:59.489000,...,2116.522,4.5,7.0,2.25,2021-01-18,2021-01-18,3.375,2021-01-25,3.15,9.45
8,24,5f77a902fc647a327b77004a,1993.0,United Kingdom,United Kingdom,DATA EXPIRED,United Kingdom,Female,APPROVED,2021-01-18 17:54:59.440000,...,2115.673,3.0,0.0,2.25,2021-01-18,2021-01-18,2.250,2021-01-18,3.15,7.35
9,25,5cf0ccf488abf10019fb775d,2001.0,New Zealand,United Kingdom,Twi,United Kingdom,Female,APPROVED,2021-01-18 17:55:09.667000,...,1913.891,4.5,90.0,2.25,2021-01-18,2021-01-18,3.375,2021-04-18,3.15,9.45


# Export modified prolific data

In [23]:
data_pay.to_csv('C:/Users/User/Google Drive/Masterarbeit/Acquise/data_pay.csv', index=False, header=True)